In [21]:
#import module

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator

### Initializing CNN

In [22]:
classifier = Sequential() # Empty Model

In [23]:
classifier.add(Conv2D(32, (3, 3), input_shape = (128, 128, 3), activation = "relu")) # 1st Hidden Layer

In [24]:
classifier.add(MaxPooling2D(pool_size = (2, 2))) 

In [25]:
classifier.add(Conv2D(64, (3, 3), activation = "relu"))# 2nd Hidden Layer

In [26]:
classifier.add(MaxPooling2D(pool_size = (2, 2))) 

In [27]:
classifier.add(Conv2D(128, (3, 3), activation = "relu"))# 3rd Hidden Layer

In [28]:
classifier.add(MaxPooling2D(pool_size = (2, 2))) 

In [29]:
classifier.add(Flatten()) # 1d Array

### Connected Layer

In [30]:
classifier.add(Dense(128, activation = 'relu'))

In [31]:
classifier.add(Dense(3, activation = 'softmax'))

In [32]:
classifier.compile(optimizer='adam',
                  loss="categorical_crossentropy",
                  metrics=['accuracy'])

### Fitting Images with the model

In [33]:
train_datagen = ImageDataGenerator(rescale=1./255, 
                                  shear_range = 0.2,
                                  zoom_range = 0.2, 
                                  horizontal_flip=True)

In [34]:
test_datagen = ImageDataGenerator(rescale=1./255)

In [15]:
training_dataset = train_datagen.flow_from_directory("dataset/training_set", target_size = (128, 128), batch_size=32, class_mode = "categorical")
test_dataset = test_datagen.flow_from_directory("dataset/test_set",target_size = (128, 128), batch_size =32, class_mode = "categorical")

Found 2400 images belonging to 3 classes.
Found 600 images belonging to 3 classes.


In [35]:
classifier.fit(training_dataset, steps_per_epoch=800/32, epochs=25, validation_data=test_dataset, validation_steps= 200/32)

Epoch 1/25
25/25 [==============================] - 18s 724ms/step - loss: 1.2320 - accuracy: 0.3288 - val_loss: 1.0936 - val_accuracy: 0.4018
Epoch 2/25
25/25 [==============================] - 17s 686ms/step - loss: 1.0861 - accuracy: 0.3963 - val_loss: 1.0536 - val_accuracy: 0.4732
Epoch 3/25
25/25 [==============================] - 17s 684ms/step - loss: 1.0311 - accuracy: 0.4588 - val_loss: 0.9744 - val_accuracy: 0.5089
Epoch 4/25
25/25 [==============================] - 17s 662ms/step - loss: 0.9817 - accuracy: 0.5000 - val_loss: 0.9088 - val_accuracy: 0.5223
Epoch 5/25
25/25 [==============================] - 17s 662ms/step - loss: 0.9186 - accuracy: 0.5275 - val_loss: 0.8545 - val_accuracy: 0.5402
Epoch 6/25
25/25 [==============================] - 17s 669ms/step - loss: 0.9536 - accuracy: 0.4963 - val_loss: 0.8619 - val_accuracy: 0.6205
Epoch 7/25
25/25 [==============================] - 17s 672ms/step - loss: 0.8663 - accuracy: 0.5875 - val_loss: 0.7988 - val_accuracy: 0.6205

In [36]:
import os 
target_dirctory = "dataset/model/"
if not os.path.exists(target_dirctory):
    os.mkdir(target_dirctory)

In [37]:
classifier.save("dataset/model/model.h5")
classifier.save_weights("dataset/model/weights.h5")

In [38]:
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.models import Sequential, load_model
from PIL import Image, ImageTk
from io import BytesIO
import requests
from tkinter import Tk, Label,Canvas, NW, Entry, Button
from keras.preprocessing import image as image_utils

In [39]:
#Load Model
image_width, image_height = 128, 128
model_path = "dataset/model/model.h5"
model_wieght_path = "dataset/model/weights.h5"
model = load_model(model_path)
model.load_weights(model_wieght_path)

In [48]:
url = ''
window = Tk()
window.title("Image Classification")
window.geometry("800x600")
lbl = Label(window, text = "Please Enter The Image Url", font = ("Halvetica", 16))
lbl.pack()


def Enter():
    global url
    lbl.configure()
    url = (User_input.get())
    print(url)
    
    
    response = requests.get(url)
    test_image = Image.open(BytesIO(response.content))
    put_image = test_image.resize((400, 400))
    test_image = test_image.resize((128, 128))
    
    img = ImageTk.PhotoImage(put_image)
    pic = Label(image = img)
    pic.pack()
    pic.image = img
    test_image = image_utils.img_to_array(test_image)
    test_image = np.expand_dims(test_image, axis = 0)
    
    result = model.predict_on_batch(test_image)
    
    
    if result[0][0] == 1:
        ans = "French Fries"
    elif result[0][1] == 1:
        ans = "Pizza"
    elif result[0][2] == 1:
        ans =  "samosa"
        
    out = Label(window, text = "Predicted Results: " + ans, font = ("Halvetica", 16))
    
    out.pack()
    
    

In [49]:
User_input = Entry(width = 100)
User_input.pack()
button = Button(window, text = "Detect the Image", font = ("Halvetica", 16), command = Enter)
button.pack()
window.mainloop()

https://images.immediate.co.uk/production/volatile/sites/30/2021/03/French-fries-b9e3e0c.jpg
